### Analyse de Subway_in  :
**Code Bokeh qui fonctionne pas bien. Il faudrait faire un checkbox en menu déroulant, pas tout afficher d'un coup, un bouton pour tout désactiver car 40 plot c'est long. Rajouter un 'hover', etc...**

In [1]:
import sys
import os

# Obtenir le chemin du dossier parent
current_path = notebook_dir = os.getcwd()
# current_path = os.path.dirname()
parent_dir = os.path.abspath(os.path.join(current_path, '..'))

# Ajouter le dossier parent au chemin de recherche des modules
if parent_dir not in sys.path:
    sys.path.insert(0, parent_dir)

In [29]:
import torch
import torch.nn as nn 
class RNN(nn.Module):
    def __init__(self,c_in,h_dim, num_layers,bias = True,batch_first = True,dropout = 0.0,bidirectional = False,lstm = True):
        super().__init__()

        # Parameters
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.lstm = lstm
        self.h_dim = h_dim

        self.rnn = nn.LSTM(input_size = c_in, hidden_size = h_dim, num_layers=num_layers,bias=bias,batch_first=batch_first,dropout=dropout,bidirectional=bidirectional)
        self.D =  2 if bidirectional else 1

    def init_hidden(self,batch_size):
        # Does not care about "batch_first"
        if self.lstm:
            h0 = torch.zeros(self.D*self.num_layers, batch_size, self.h_dim)
            c0 = torch.zeros(self.D*self.num_layers, batch_size, self.h_dim)
            return(h0,c0)

    def forward(self,x):
        ''' x.shape : [B,C,N,L]
        
        has to be transformed in [B',L,C] to return [B',L,D*C']  
        Where D = 2 if bidirectional else 1
        '''
        B,C,N,L = x.shape
        x = x.permute(0,2,3,1)
        x = x.reshape(-1,x.size(2),x.size(3))
        permute_back = True

        # Init hidden state
        h0 = self.init_hidden(x.size(0) if self.batch_first else x.size(1))

        # Rnn
        print(x.size(), 'supposed to be [B*N, L, C_in]')  
        print(h0[0].size(),h0[1].size(), len(h0), 'supposed to be [num_layer*D, B*N, h_dim]')
        print(self.rnn,'supposed to be [c_in, h_dim, num_layer]')  

        x, (hn1,hn2) = self.rnn(x,(h0[0],h0[1]))  #[B,L,D*H_dim]   

        return(x)
    


In [30]:
import torch.nn as nn
import torch
c_in  = 1
B = 8
N = 10
L = 6
h_dim = 10
num_layers = 2
bidirectional = True

lstm = RNN(c_in,h_dim, num_layers,bidirectional=bidirectional)

input = torch.randn(B,c_in,N,L)
out= lstm(input)

print(out.size())
print(h1.size(),c1.size())

torch.Size([80, 6, 1]) supposed to be [B*N, L, C_in]
torch.Size([4, 80, 10]) torch.Size([4, 80, 10]) 2 supposed to be [num_layer*D, B*N, h_dim]
LSTM(1, 10, num_layers=2, batch_first=True, bidirectional=True) supposed to be [c_in, h_dim, num_layer]
torch.Size([80, 6, 20])
torch.Size([2, 80, 10]) torch.Size([2, 80, 10])


In [15]:
rnn = nn.LSTM(10, 20, 2,batch_first=True)
B = 3
input = torch.randn(B,5, 10)
h0 = torch.randn(2, B, 20)
c0 = torch.randn(2, B, 20)
output, (hn, cn) = rnn(input, (h0, c0))

print(output.size())
print(hn.size(),cn.size())

torch.Size([3, 5, 20])
torch.Size([2, 3, 20]) torch.Size([2, 3, 20])


In [6]:
from build_inputs.load_adj import load_adj
import numpy as np
import scipy.sparse as sp
from scipy.sparse import linalg


adj_mx,num_nodes = load_adj(adj_type = 'adj')
sp_adj_mx = sp.coo_matrix(adj_mx.T)
d = np.array(sp_adj_mx.sum(1))

In [3]:
import numpy as np 

from bokeh.plotting import figure, show, output_notebook, output_file, save
from bokeh.models import Legend, LegendItem,HoverTool
from bokeh.palettes import Turbo256
from bokeh.layouts import row

# Personnal Import 
from utils.utilities_DL import load_raw_data
from dataset import DataSet
# ...

# Paths
FOLDER_PATH = 'data/'

#Init and load data: 
H,W,D = 6,1,1
L = H+W+D 
step_ahead = 1

window_pred = np.arange(2*96)

FILE_NAME = 'subway_IN_interpol_neg_15_min_2019_2020.csv'



In [4]:
# Load df, remove forbidden values: 
df,invalid_dates,time_step_per_hour = load_raw_data(FOLDER_PATH,FILE_NAME,single_station = False)
dataset = DataSet(df,time_step_per_hour=time_step_per_hour, Weeks = W, Days = D, historical_len= H,step_ahead=step_ahead)
dataset.get_shift_between_set()   # get shift indice and shift date from the first element / between each dataset 
dataset.get_feature_vect()  # Build 'df_shifted'.
dataset.remove_forbidden_prediction(invalid_dates) # Build 'df_verif' , which is df_shifted without sequences which contains invalid date

# Find atypical Value. Check if we need to modify them.
valid_df = dataset.df[~(dataset.df.index.isin(invalid_dates))]
limit = 1500
all_plot = False

p = figure(x_axis_type='datetime', height=600, width=1400, sizing_mode="scale_width")

# Stocker les renderers pour les ajouter à la légende
renderers = []

if all_plot:
    for i, column in enumerate(df.columns):
        x, y = df.index, df[column]
        color = Turbo256[int(256 / len(df.columns) * i)]  
        renderer = p.line(x, y, line_width=2, color=color, alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)
        renderer.visible = False
        renderers.append(renderer)

    legend = Legend(items=[LegendItem(label=col, renderers=[rend], index=i) for i, (col, rend) in enumerate(zip(df.columns, renderers))])

else:

    for i,column in enumerate(valid_df):
        color = Turbo256[int(256 / len(df.columns) * i)]  
        sub_df = valid_df[valid_df[column] > limit]
        if not sub_df.empty:
            x,y = sub_df.index,sub_df[column]
            renderer = p.scatter(x, y, color=color, alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)
            renderers.append(renderer)

    x,y = dataset.df.index,dataset.df.quantile(0.9,axis = 1)
    renderer = p.line(x, y, line_width=2, color='black', alpha=0.8, legend_label='all_stations', muted_color=color, muted_alpha=0)
    renderers.append(renderer)

    legend = Legend(items=[LegendItem(label=col, renderers=[rend], index=i) for i, (col, rend) in enumerate(zip(list(df.columns)+['all_stations'], renderers))])  


p.add_layout(legend, 'right')
p.legend.click_policy = "mute"
hover = HoverTool()
p.add_tools(hover)
output_notebook()
show(p) 

saving = False
if saving:
    output_file("df_IN_outliers.html")
save(p)

TypeError: load_raw_data() missing 1 required positional argument: 'FILE_NAME'

In [ ]:
p = figure(x_axis_type='datetime', height=600, width=1400, sizing_mode="scale_width")

column = 'Ampère Victor Hugo'
column = 'Bellecour'
x,y = dataset.df.index,dataset.df[column]

renderer = p.line(x, y, color='blue', alpha=0.8, legend_label=column, muted_color=color, muted_alpha=0)

output_notebook() 
show(p)

output_file(f"df_IN_{column}.html")
save(p)
